<a href="https://colab.research.google.com/github/akshat7gupta/deep-learning/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/


/content/drive/MyDrive


In [ ]:
!ls

sample_data


In [ ]:
from keras.applications import ResNet50
# load the model
model = ResNet50()
model.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                    

In [ ]:


#Let us load an image to test the pretrained VGG model.
#These models are developed on powerful computers so we may as well use them for transfer learning
#For VGG16 the images need to be 224x224.
from keras.preprocessing.image import load_img
image = load_img('/content/drive/MyDrive/dlimg.jpg', target_size=(224, 224))

#Convert pixels to Numpy array
from keras.preprocessing.image import img_to_array
image = img_to_array(image)

# Reshape data for the model. VGG expects multiple images of size 224x224x3,
#therefore the input shape needs to be (1, 224, 224, 3)
#image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
import numpy as np
image = np.expand_dims(image, axis=0)

#Data needs to be preprocessed same way as the training dataset, to get best results
#preprocessing from Keras does this job.
#Notice the change in pixel values (Preprocessing subtracts mean RGB value of training set from each pixel)
from keras.applications.vgg16 import preprocess_input
image = preprocess_input(image)


# predict the probability across all output categories.
#Probability for each of the 1000 classes will be calculated.
pred = model.predict(image)

#Print the probabilities of the top 5 classes
from tensorflow.keras.applications.mobilenet import decode_predictions
pred_classes = decode_predictions(pred, top=5)
for i in pred_classes[0]:
    print(i)

35363/35363 [==============================] - 0s 0us/step
('n04069434', 'reflex_camera', 0.88629544)
('n03657121', 'lens_cap', 0.034179624)
('n02841315', 'binoculars', 0.014989514)
('n04328186', 'stopwatch', 0.014895703)
('n04485082', 'tripod', 0.0113934195)


In [ ]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

In [ ]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16()

553467096/553467096 [==============================] - 6s 0us/step


In [ ]:
newmodel = Sequential()
#num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)
newmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
for layer in newmodel.layers:
  layer.trainable=False   #We don't want to train these layers again, so False.

In [ ]:
path = 'images/colorization/'
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train = train_datagen.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode=None)

Found 0 images belonging to 0 classes.


In [ ]:
#Convert from RGB to Lab
"""
by iterating on each image, we convert the RGB to Lab.
Think of LAB image as a grey image in L channel and all color info stored in A and B channels.
The input to the network will be the L channel, so we assign L channel to X vector.
And assign A and B to Y.

"""

X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128,
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

ValueError: ignored

In [ ]:
#now we have one channel of L in each layer but, VGG16 is expecting 3 dimension,
#so we repeated the L channel two times to get 3 dimensions of the same L channel

vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(0,)


In [ ]:
#Decoder
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         295040    
                                                                 
 up_sampling2d (UpSampling2  (None, 14, 14, 128)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 up_sampling2d_1 (UpSamplin  (None, 28, 28, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)       

In [ ]:
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=10, batch_size=128)

model.save('colorize_autoencoder_VGG16.model')

ValueError: ignored

In [ ]:
############################################
#Predicting using saved model.
model = tf.keras.models.load_model('colorize_autoencoder_VGG16_10000.model',
                                   custom_objects=None,
                                   compile=True)

OSError: ignored

In [ ]:
testpath = 'images/colorization2/'
files = os.listdir(testpath)
for idx, file in enumerate(files):
    test = img_to_array(load_img(testpath+file))
    test = resize(test, (224,224), anti_aliasing=True)
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    #print(L.shape)
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    #print(ab.shape)
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    imsave('images/colorization2/vgg_result/result'+str(idx)+".jpg", lab2rgb(cur))